In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import json
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

In [2]:
filename_list = []
data = []
targets = []

with open("imgs_preprocessed/export-2021-04-14T17-38-24.110Z.json") as json_file:
    json_data = json.load(json_file)
    for elem in json_data:
        filename = elem['External ID']
        output_mask = np.zeros((280,1,2))
        if 'objects' in elem['Label']:
             for label in elem['Label']['objects']:
                 if label['title'] == 'level':
                     if 'bbox' in label:
                        y0 = label['bbox']['top']
                        h = label['bbox']['height']
                        y1 = y0+h
                        output_mask[y0-2:y1-2,0,0] = 1
                 if label['title'] == 'foam':
                     y0 = label['bbox']['top']
                     h = label['bbox']['height']
                     y1 = y0+h
                     output_mask[y0:y1,0,1] = 1
        targets.append(output_mask[2:278,:,:])

        image = load_img('imgs_preprocessed/gray/%s' % filename)
        image = img_to_array(image)
        image = image[:,:,0]
        image = np.expand_dims(image, axis=2) #adding a color dimension
        data.append(image)

        filename_list.append(filename)

In [4]:
filename_list = []
data = []
targets = []

with open("imgs_preprocessed/export-2021-03-27T14-16-42.456Z.json") as json_file:
    json_data = json.load(json_file)
    for elem in json_data:
        filename = elem['External ID']
        output_mask = np.zeros((280,1,2))
        if 'objects' in elem['Label']:
             for label in elem['Label']['objects']:
                 if label['title'] == 'level':
                     y = label['point']['y']
                     y = y-2
                     output_mask[y,0,0] = 1
                 if label['title'] == 'foam':
                     y0 = label['bbox']['top']
                     h = label['bbox']['height']
                     y1 = y0+h
                     output_mask[y0:y1,0,1] = 1
        targets.append(output_mask[2:278,:,:])

        image = load_img('imgs_preprocessed/%s' % filename)
        image = img_to_array(image)
        image = image[:,:,0]
        image = np.expand_dims(image, axis=2) #adding a color dimension
        #image = np.expand_dims(image, axis=0) #adding a batch dimension
        data.append(image)

        filename_list.append(filename)

In [5]:
data = np.array(data, dtype='float32') / 255
targets = np.array(targets, dtype='float32')

split = train_test_split(data, targets, filename_list, test_size=0.1, random_state=42)
(train_imgs, test_imgs) = split[:2]
(train_targets, test_targets) = split[2:4]
(train_filenames, test_filenames) = split[4:]

In [6]:
model = keras.Sequential(
    [
        keras.Input(shape=(280,40,1)),
        layers.Conv2D(16, (5, 40), strides=1, padding='valid', activation="relu"),
        layers.Conv2D(16, (7, 1), padding='same', activation="relu"),
        layers.Conv2D(16, (5, 1), padding='same', activation="relu"),
        layers.Conv2D(16, (3, 1), padding='same', activation="relu"),
        layers.Conv2D(2, 1, padding='same', activation='sigmoid')
    ]
)

In [7]:
opt = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss='mse', optimizer=opt)

In [10]:
class CustomCallback(keras.callbacks.Callback):
    def __init__(self, a):
        super().__init__()
        print(a)
    def on_train_epoch_end(self, epoch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(epoch, logs["loss"]))


In [11]:
H = model.fit(
    train_imgs,train_targets,
    validation_data=(test_imgs,test_targets),
    batch_size=10,
    epochs=100,
    verbose=1,
    callbacks=[CustomCallback('42')]
)

42
Train on 70 samples, validate on 8 samples
Epoch 1/100
70/70 [==============================] - 0s 5ms/sample - loss: 0.0270 - val_loss: 0.0202
Epoch 2/100
70/70 [==============================] - 0s 5ms/sample - loss: 0.0203 - val_loss: 0.0179
Epoch 3/100
70/70 [==============================] - 0s 5ms/sample - loss: 0.0183 - val_loss: 0.0164
Epoch 4/100
70/70 [==============================] - 0s 6ms/sample - loss: 0.0168 - val_loss: 0.0174
Epoch 5/100
70/70 [==============================] - 0s 6ms/sample - loss: 0.0169 - val_loss: 0.0139
Epoch 6/100
70/70 [==============================] - 0s 6ms/sample - loss: 0.0191 - val_loss: 0.0180
Epoch 7/100
70/70 [==============================] - 0s 6ms/sample - loss: 0.0189 - val_loss: 0.0298
Epoch 8/100
70/70 [==============================] - 0s 6ms/sample - loss: 0.0255 - val_loss: 0.0138
Epoch 9/100
70/70 [==============================] - 0s 6ms/sample - loss: 0.0190 - val_loss: 0.0269
Epoch 10/100
70/70 [=========================

In [12]:
model.save('weight_gray_3.h5', save_format="h5")